A short script to convert a flight path file from .csv to the .DEG format expected by CARI-7


In [209]:
from datetime import datetime
import numpy as np
import time


In [210]:
waypoints = np.genfromtxt("./Flight_QF64.csv", \
            dtype = "i8, S50, S50, S50, S50, f8, f8, f8", \
            names = ['timestamp', 'UTC', 'Callsign', 'Latitude', 'Longitude', 'Altitude', 'Speed', 'Direction'], \
            delimiter = ",")


In [211]:
start_date=datetime.utcfromtimestamp(waypoints[0]['timestamp'])

In [212]:
header = np.str(start_date.month)+"/"+np.str(start_date.year)+", "+waypoints[0]['Callsign'].decode("utf-8")+"\n"


In [213]:
f = open('Flight_QF64.DEG', 'wb')

In [214]:
f.write(header.encode('utf8'))

14

In [215]:
header = "DEG MIN N/S DEG MIN E/W FEET TIME(MIN) \n"

In [216]:
f.write(header.encode('utf8'))

40

In [217]:
start_datetime = datetime.utcfromtimestamp(waypoints[0]['timestamp'])

In [218]:
for wp in waypoints:
    #calculate time for this waypoint
    datetime = datetime.utcfromtimestamp(wp['timestamp'])
    #calculate time since last waypoint (will be zero for first one)
    delta_datetime = datetime - start_datetime
    delta_mins = np.int(delta_datetime.seconds/60) # expected by CARI-7
    
    #get the latitude as a string and remove the quotation mark
    latitude = wp['Latitude'].decode("utf-8").replace('"','')
    #convert to float and strip the degrees
    lat_degs = np.int(float(latitude))
    #the remainder will be fraction of degree, convert from decimal to minutes
    lat_minsecs = (float(latitude) - lat_degs)*60
    #determine the bearing, positive lat is North, negative is South
    lat_bear = 'N'
    if lat_degs < 0 : lat_bear = 'S'
        
    #repeat for longitude
    longitude = wp['Longitude'].decode("utf-8").replace('"','')
    #convert to float and strip the degrees
    long_degs = np.int(float(longitude))
    #the remainder will be fraction of degree, convert from decimal to minutes
    long_minsecs = (float(longitude) - long_degs)*60
    #determine the bearing, positive long is East, negative is West
    long_bear = 'E'
    if long_degs < 0 : long_bear = 'W'
        
    entry = np.str(np.abs(lat_degs)) + ", " + np.str(np.abs(lat_minsecs)) + ", " + lat_bear + ", " + \
        np.str(np.abs(long_degs)) + ", " + np.str(np.abs(long_minsecs)) + ", " + long_bear + ", " + \
        np.str(wp['Altitude']) + ", " + np.str(delta_mins) + "\n"
        
    f.write(entry.encode('utf8'))
        
        
    
    
    

In [219]:
f.close()    
